In [2]:
import numpy as np 
import pandas as pd
import pickle


# Data
- Load data from pickle file
- split the data into independent and dependent 
- split to train and test set 

In [3]:
data = pickle.load(open('./data_face_features.pickle', mode='rb'))

In [4]:
x = np.array(data['data'])# indendepnd variable
y = np.array(data['label']) # dependt varibale

In [5]:
x.shape , y.shape

((1654, 1, 128), (1654,))

In [6]:
x = x.reshape(-1,128)
x.shape

(1654, 128)

In [7]:
# split the data into train and test
from sklearn.model_selection import train_test_split

In [8]:
x_train,x_test, y_train,y_test = train_test_split(x,y,train_size=0.8,random_state=0)

In [9]:
x_train.shape, x_test.shape,y_train.shape, y_test.shape

((1323, 128), (331, 128), (1323,), (331,))

# Train Machine Learing 

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, accuracy_score,f1_score

### Logistic Regression 

In [11]:
model_logistic = LogisticRegression()
model_logistic.fit(x_train,y_train) #training logistic Regression

LogisticRegression()

In [12]:
def get_report(model,x_train,y_train,x_test,y_test):
    y_pred_train = model.predict(x_train)
    y_pred_test = model.predict(x_test)

    # accuracy score
    acc_train = accuracy_score(y_train,y_pred_train)
    acc_test = accuracy_score(y_test,y_pred_test)

    # f1 score 
    f1_score_train = f1_score(y_train,y_pred_train,average='macro')
    f1_score_test = f1_score(y_test,y_pred_test,average='macro')
    
    print('Accuracy Train = %0.2f' %acc_train)
    print('Accuracy test = %0.2f' %acc_test)
    print('F1 Score Train = %0.2f' %f1_score_train)
    print('F1 Score Test = %0.2f' %f1_score_test)

In [13]:
get_report(model_logistic,x_train,y_train,x_test,y_test)

Accuracy Train = 0.73
Accuracy test = 0.67
F1 Score Train = 0.73
F1 Score Test = 0.65


# Support Vector Machines

In [14]:
model_svc = SVC(probability=True)
model_svc.fit(x_train,y_train)

SVC(probability=True)

In [15]:
get_report(model_svc,x_train,y_train,x_test,y_test)

Accuracy Train = 0.84
Accuracy test = 0.69
F1 Score Train = 0.84
F1 Score Test = 0.68


# Random Forest Classifier

In [18]:
model_rf = RandomForestClassifier(n_estimators=10)
model_rf.fit(x_train,y_train)

RandomForestClassifier(n_estimators=10)

In [19]:
get_report(model_rf,x_train,y_train,x_test,y_test)

Accuracy Train = 1.00
Accuracy test = 0.59
F1 Score Train = 1.00
F1 Score Test = 0.58


# Voting Classifier

In [23]:
model_voting = VotingClassifier(estimators=[
    ('logistic',LogisticRegression()),
    ('svc',SVC(probability=True)),
    ('rf',RandomForestClassifier())
],voting='soft',weights=[2,3,1])

In [24]:
model_voting.fit(x_train,y_train)

VotingClassifier(estimators=[('logistic', LogisticRegression()),
                             ('svc', SVC(probability=True)),
                             ('rf', RandomForestClassifier())],
                 voting='soft', weights=[2, 3, 1])

In [25]:
get_report(model_voting,x_train,y_train,x_test,y_test)

Accuracy Train = 0.88
Accuracy test = 0.69
F1 Score Train = 0.88
F1 Score Test = 0.68


# Parameter Tuning

In [26]:
from sklearn.model_selection import GridSearchCV

In [27]:
model_grid = GridSearchCV(model_voting,
                          param_grid={
                              'svc__C':[3,5,7,10],
                              'svc__gamma':[0.1,0.3,0.5],
                              'rf__n_estimators':[5,10,20],
                              'rf__max_depth':[3,5,7],
                              'voting':['soft','hard']
                          },scoring='accuracy',cv=3,n_jobs=1,verbose=2)

In [28]:
model_grid.fit(x_train,y_train)

Fitting 3 folds for each of 216 candidates, totalling 648 fits
[CV] END rf__max_depth=3, rf__n_estimators=5, svc__C=3, svc__gamma=0.1, voting=soft; total time=   0.3s
[CV] END rf__max_depth=3, rf__n_estimators=5, svc__C=3, svc__gamma=0.1, voting=soft; total time=   0.3s
[CV] END rf__max_depth=3, rf__n_estimators=5, svc__C=3, svc__gamma=0.1, voting=soft; total time=   0.2s
[CV] END rf__max_depth=3, rf__n_estimators=5, svc__C=3, svc__gamma=0.1, voting=hard; total time=   0.2s
[CV] END rf__max_depth=3, rf__n_estimators=5, svc__C=3, svc__gamma=0.1, voting=hard; total time=   0.3s
[CV] END rf__max_depth=3, rf__n_estimators=5, svc__C=3, svc__gamma=0.1, voting=hard; total time=   0.3s
[CV] END rf__max_depth=3, rf__n_estimators=5, svc__C=3, svc__gamma=0.3, voting=soft; total time=   0.2s
[CV] END rf__max_depth=3, rf__n_estimators=5, svc__C=3, svc__gamma=0.3, voting=soft; total time=   0.2s
[CV] END rf__max_depth=3, rf__n_estimators=5, svc__C=3, svc__gamma=0.3, voting=soft; total time=   0.2s
[

GridSearchCV(cv=3,
             estimator=VotingClassifier(estimators=[('logistic',
                                                     LogisticRegression()),
                                                    ('svc',
                                                     SVC(probability=True)),
                                                    ('rf',
                                                     RandomForestClassifier())],
                                        voting='soft', weights=[2, 3, 1]),
             n_jobs=1,
             param_grid={'rf__max_depth': [3, 5, 7],
                         'rf__n_estimators': [5, 10, 20],
                         'svc__C': [3, 5, 7, 10], 'svc__gamma': [0.1, 0.3, 0.5],
                         'voting': ['soft', 'hard']},
             scoring='accuracy', verbose=2)

In [30]:
model_best_estimator = model_grid.best_estimator_

In [31]:
model_grid.best_score_

0.709750566893424

# Save Model 

In [32]:
pickle.dump(model_best_estimator,open('../models/machinelearning_face_person_identity.pkl',mode='wb'))